In [1]:
# Datasets grabbed from https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
# Let's look at the problem of forecasting stocks

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from data import picture, dataShift, get_x_y, get_mdl_inputs, calc_y, get_trn_val

In [ ]:
# Let's import one of the datasets from the link.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('aadr.us.csv')

numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']


In [ ]:
draw = False
if draw == True:
    picture(df)

In [ ]:
cleaner = dataShift(df, numeric_cols)

In [ ]:
x, y, past_cols = get_x_y(numeric_cols, cleaner)

In [ ]:
y_diff, x_with_current = get_mdl_inputs(cleaner, numeric_cols, past_cols)

In [ ]:

x_with_current.shape, y_diff.shape

In [ ]:
# mdl = Ridge().fit(x_with_current, y_diff)
# yhat = mdl.predict(x_with_current)
# mdl = Ridge().fit(x, y)
# yhat = mdl.predict(x)


# predictive_correlation = np.corrcoef(yhat, y_diff)
# predictive_correlation[1,0]

# treemdl = GradientBoostingRegressor().fit(x_with_current, y_diff)
# treeyhat = treemdl.predict(x_with_current)

# predictive_correlation_tree = np.corrcoef(treeyhat, y_diff)
# predictive_correlation_tree[1,0]


# nnmdl = MLPRegressor([100]).fit(x_with_current, y_diff)
# nnyhat = nnmdl.predict(x_with_current)

# predictive_correlation_nn = np.corrcoef(nnyhat, y_diff)
# predictive_correlation_nn[1,0]

In [ ]:
x_trn, y_trn, x_val, y_val = get_trn_val(x_with_current, numeric_cols, 0.6)

In [ ]:
mdl = GradientBoostingRegressor().fit(x_trn, y_trn)
yhat = mdl.predict(x_val)
predictive_correlation = np.corrcoef(yhat, y_val)
predictive_correlation[1,0]


In [ ]:
mdl = Ridge().fit(x_trn, y_trn)
yhat = mdl.predict(x_trn)
predictive_correlation = np.corrcoef(yhat, y_trn)
predictive_correlation[1,0]

In [ ]:
predictive_correlation

In [ ]:
def validation(mdl,x_val,y_val):
    money = 100
    predictions = mdl.predict(x_val)
    assignments = predictions > predictions.mean()
    for assignment,y in zip(assignments, y_val):
        money = money * (1-assignment) + assignment * money * (1 + y)
    return money

In [ ]:
validation(mdl,x_val,y_val)

In [ ]:
def baseline(y_val):
    money = 100
    for y in y_val:
        money = money * (1+y) 
    return money

In [ ]:
baseline(y_val)

In [ ]:
profit =[]
validations_correlations = []
baseline_profits=[]
training_correlations =[]
for pct_train in [.2,.3,.4,.5,.6,.7,.8,.9]:
    x_trn, y_trn, x_val, y_val  = get_trn_val(x_with_current, numeric_cols, pct_train, val_percent = 0.1)
    mdl = Ridge().fit(x_trn, y_trn)
    yhat = mdl.predict(x_trn)
    predictive_correlation = np.corrcoef(yhat, y_trn)
    training_correlations.append(predictive_correlation[1,0])
    mdl_val = Ridge().fit(x_val, y_val)
    yhat_val = mdl.predict(x_val)
    baseline_profits.append(baseline(y_val))
    predictive_correlation_val = np.corrcoef(yhat_val, y_val)
    validations_correlations.append(predictive_correlation_val[1,0])
    profit.append(validation(mdl,x_val, y_val))

In [ ]:
training_correlations

In [ ]:
sum(profit)/len(profit)

In [ ]:
validations_correlations

In [ ]:
sum(baseline_profits)/len(baseline_profits)

In [ ]:
x_trn.shape